In [17]:
import os
import random
import pandas as pd
import matplotlib.pyplot as plt
import torch, torchaudio
import torch.nn as nn
import torch.utils.data as data
import torch.nn.functional as F
from torch.utils.data import Dataset
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [19]:
class NepaliSoundDataset(Dataset):
    def __init__(
        self,
        annotations_file_path,
        audio_dir,
    ):
        self.annotaions = pd.read_csv(annotations_file_path, sep="\t")
        self.audio_dir = audio_dir        

    def __len__(self):
        return len(self.annotaions)

    def _get_speaker_id(self, index):
        return self.annotaions.iloc[index, 0].strip()

    def _get_audio_sample_path(self, index):
        filename = self.annotaions.iloc[index, 0].strip()
        path = os.path.join(self.audio_dir, filename + ".wav")
        return path

    def _get_audio_sample_output(self, index):
        return self.annotaions.iloc[index, 1]

    def __getitem__(self, index):
        audio_sample_path = self._get_audio_sample_path(index)
        output = self._get_audio_sample_output(index)
        signal, sr = torchaudio.load(audio_sample_path, normalize=True)
        speaker_id = self._get_speaker_id(index)
        return (signal, sr, output, speaker_id)

In [20]:
class SpeechRecognitionModel(nn.Module):
    def __init__(self, input_dim=13, num_classes=77, cnn_output_dim=200, gru_hidden_dim=256):
        super(SpeechRecognitionModel, self).__init__()
        
        # CNN Layer
        self.conv1 = nn.Conv1d(input_dim, cnn_output_dim, kernel_size=11, stride=1, padding=5)
        self.bn1 = nn.BatchNorm1d(cnn_output_dim)
        
        # GRU Layer
        self.gru = nn.GRU(cnn_output_dim, gru_hidden_dim, num_layers=1, bidirectional=True, batch_first=True)
        self.bn2 = nn.BatchNorm1d(gru_hidden_dim * 2)
        
       # Dense Layer
        self.fc = nn.Linear(gru_hidden_dim * 2, num_classes)

        # Fully connected layer
        self.fc = nn.Linear(gru_hidden_dim * 2, num_classes)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = x.permute(0, 2, 1)  # (batch_size, seq_length, cnn_output_dim)
        x, _ = self.gru(x)
        x = x.permute(0, 2, 1)  # (batch_size, gru_hidden_dim * 2, seq_length)
        x = self.bn2(x)
        x = x.permute(0, 2, 1)  # (batch_size, seq_length, gru_hidden_dim * 2)
        x = self.fc(x)
        return x

In [21]:
import numpy as np


def avg_wer(wer_scores, combined_ref_len):
    return float(sum(wer_scores)) / float(combined_ref_len)


def _levenshtein_distance(ref, hyp):
    """Levenshtein distance is a string metric for measuring the difference
    between two sequences. Informally, the levenshtein disctance is defined as
    the minimum number of single-character edits (substitutions, insertions or
    deletions) required to change one word into the other. We can naturally
    extend the edits to word level when calculate levenshtein disctance for
    two sentences.
    """
    m = len(ref)
    n = len(hyp)

    # special case
    if ref == hyp:
        return 0
    if m == 0:
        return n
    if n == 0:
        return m

    if m < n:
        ref, hyp = hyp, ref
        m, n = n, m

    # use O(min(m, n)) space
    distance = np.zeros((2, n + 1), dtype=np.int32)

    # initialize distance matrix
    for j in range(0, n + 1):
        distance[0][j] = j

    # calculate levenshtein distance
    for i in range(1, m + 1):
        prev_row_idx = (i - 1) % 2
        cur_row_idx = i % 2
        distance[cur_row_idx][0] = i
        for j in range(1, n + 1):
            if ref[i - 1] == hyp[j - 1]:
                distance[cur_row_idx][j] = distance[prev_row_idx][j - 1]
            else:
                s_num = distance[prev_row_idx][j - 1] + 1
                i_num = distance[cur_row_idx][j - 1] + 1
                d_num = distance[prev_row_idx][j] + 1
                distance[cur_row_idx][j] = min(s_num, i_num, d_num)

    return distance[m % 2][n]


def word_errors(reference, hypothesis, ignore_case=False, delimiter=" "):
    """Compute the levenshtein distance between reference sequence and
    hypothesis sequence in word-level.
    :param reference: The reference sentence.
    :type reference: basestring
    :param hypothesis: The hypothesis sentence.
    :type hypothesis: basestring
    :param ignore_case: Whether case-sensitive or not.
    :type ignore_case: bool
    :param delimiter: Delimiter of input sentences.
    :type delimiter: char
    :return: Levenshtein distance and word number of reference sentence.
    :rtype: list
    """
    if ignore_case == True:
        reference = reference.lower()
        hypothesis = hypothesis.lower()

    ref_words = reference.split(delimiter)
    hyp_words = hypothesis.split(delimiter)

    edit_distance = _levenshtein_distance(ref_words, hyp_words)
    return float(edit_distance), len(ref_words)


def char_errors(reference, hypothesis, ignore_case=False, remove_space=False):
    """Compute the levenshtein distance between reference sequence and
    hypothesis sequence in char-level.
    :param reference: The reference sentence.
    :type reference: basestring
    :param hypothesis: The hypothesis sentence.
    :type hypothesis: basestring
    :param ignore_case: Whether case-sensitive or not.
    :type ignore_case: bool
    :param remove_space: Whether remove internal space characters
    :type remove_space: bool
    :return: Levenshtein distance and length of reference sentence.
    :rtype: list
    """
    if ignore_case == True:
        reference = reference.lower()
        hypothesis = hypothesis.lower()

    join_char = " "
    if remove_space == True:
        join_char = ""

    reference = join_char.join(filter(None, reference.split(" ")))
    hypothesis = join_char.join(filter(None, hypothesis.split(" ")))

    edit_distance = _levenshtein_distance(reference, hypothesis)
    return float(edit_distance), len(reference)


def wer(reference, hypothesis, ignore_case=False, delimiter=" "):
    """Calculate word error rate (WER). WER compares reference text and
    hypothesis text in word-level. WER is defined as:
    .. math::
        WER = (Sw + Dw + Iw) / Nw
    where
    .. code-block:: text
        Sw is the number of words subsituted,
        Dw is the number of words deleted,
        Iw is the number of words inserted,
        Nw is the number of words in the reference
    We can use levenshtein distance to calculate WER. Please draw an attention
    that empty items will be removed when splitting sentences by delimiter.
    :param reference: The reference sentence.
    :type reference: basestring
    :param hypothesis: The hypothesis sentence.
    :type hypothesis: basestring
    :param ignore_case: Whether case-sensitive or not.
    :type ignore_case: bool
    :param delimiter: Delimiter of input sentences.
    :type delimiter: char
    :return: Word error rate.
    :rtype: float
    :raises ValueError: If word number of reference is zero.
    """
    edit_distance, ref_len = word_errors(reference, hypothesis, ignore_case, delimiter)

    if ref_len == 0:
        raise ValueError("Reference's word number should be greater than 0.")

    wer = float(edit_distance) / ref_len
    return wer


def cer(reference, hypothesis, ignore_case=False, remove_space=False):
    """Calculate charactor error rate (CER). CER compares reference text and
    hypothesis text in char-level. CER is defined as:
    .. math::
        CER = (Sc + Dc + Ic) / Nc
    where
    .. code-block:: text
        Sc is the number of characters substituted,
        Dc is the number of characters deleted,
        Ic is the number of characters inserted
        Nc is the number of characters in the reference
    We can use levenshtein distance to calculate CER. Chinese input should be
    encoded to unicode. Please draw an attention that the leading and tailing
    space characters will be truncated and multiple consecutive space
    characters in a sentence will be replaced by one space character.
    :param reference: The reference sentence.
    :type reference: basestring
    :param hypothesis: The hypothesis sentence.
    :type hypothesis: basestring
    :param ignore_case: Whether case-sensitive or not.
    :type ignore_case: bool
    :param remove_space: Whether remove internal space characters
    :type remove_space: bool
    :return: Character error rate.
    :rtype: float
    :raises ValueError: If the reference length is zero.
    """
    edit_distance, ref_len = char_errors(
        reference, hypothesis, ignore_case, remove_space
    )

    if ref_len == 0:
        raise ValueError("Length of reference should be greater than 0.")

    cer = float(edit_distance) / ref_len
    return cer


class TextTransform:
    """Maps characters to integers and vice versa"""

    def __init__(self):
        char_map_str = """
        ' 0
        अ  1
        आ 2 
        इ  3 
        ई  4 
        उ  5
        ऊ  6
        ए  7 
        ऐ  8 
        ओ 9 
        औ 10
        क 11
        ख 12 
        ग 13 
        घ 14
        ङ 15 
        च 16
        छ 17
        ज 18 
        झ 19 
        ञ 20
        ट 21 
        ठ 22 
        ड 23 
        ढ 24 
        ण 25 
        त 26 
        थ 27
        द 28
        ध 29
        न 30 
        प 31 
        फ 32 
        ब 33 
        भ 34 
        म 35
        य 36
        र 37
        ल 38
        व 39
        श 40
        ष 41
        स 42
        ह 43
         ँ  44
         ं  45
         ः  46
         ्  47 
         ा  48
         ि  49 
         ी  50 
         ु  51
         ू  52
         ृ  53
         े  54
         ै  55
         ो  56
         ौ  57
        ॐ 58
        ऋ 59
        ।  60 
        ०  61
        १  62
        २  63
        ३  64
        ४  65
        ५  66
        ६  67
        ७  68
        ८  69
        ९  70
        <SPACE> 71
        \u200c 72
        \u200d 73
        . 74
         ़  75
        <UNK> 76
        """

        self.char_map = {}
        self.index_map = {}
        for line in char_map_str.strip().split("\n"):
            ch, index = line.split()
            self.char_map[ch] = int(index)
            self.index_map[int(index)] = ch
        self.index_map[71] = " "

    def text_to_int(self, text):
        """Use a character map and convert text to an integer sequence"""
        int_sequence = []
        for c in text:
            if c == " ":
                ch = self.char_map["<SPACE>"]
            else:
                ch = self.char_map.get(c, self.char_map["<UNK>"])
            int_sequence.append(ch)
        return int_sequence

    def int_to_text(self, labels):
        """Use a character map and convert integer labels to an text sequence"""
        string = []
        for i in labels:
            string.append(self.index_map[i])
        return "".join(string).replace("<SPACE>", " ")

In [22]:
SAMPLE_RATE = 16000
NO_MFCC = 13
NO_MELS = 26
NO_FFT = 256
WINDOW_LEN = int(0.025 * SAMPLE_RATE)  # 25ms window size
HOP_LEN = int(0.01 * SAMPLE_RATE)  # 10ms hop size (commonly used)
AUDIO_DIR = "/Users/santoshpandey/Desktop/ASR/code/data/OpenSLR2/wavs"
ANNOTATIONS_FILE_PATH = "/Users/santoshpandey/Desktop/ASR/code/data/OpenSLR2/line_index.tsv"

device = "cuda" if torch.cuda.is_available() else "cpu"

mfcc_spectrogram = torchaudio.transforms.MFCC(sample_rate=SAMPLE_RATE,
                                              n_mfcc=NO_MFCC,
                                              melkwargs={"n_fft": NO_FFT,
                                                         "hop_length": HOP_LEN,
                                                         "n_mels": NO_MELS,                                                         
                                                         "center": False})

def compute_mfcc(audio_filename, mfcc_transform):
    t_signal, t_sr = torchaudio.load(audio_filename, normalize=True)
    return mfcc_transform(t_signal)

def get_audio_filenames(audio_dir):
    return [os.path.join(audio_dir, _filename) for _filename in os.listdir(audio_dir)]

def calculate_average_mfccs(audio_dir, mfcc_transform):
    """
    Calculate mean and standard deviation from a sample of the dataset, 
    which helps in normalization later.
    """
    audio_filenames = get_audio_filenames(audio_dir)
    mfccs = [compute_mfcc(_filename, mfcc_transform) for _filename in audio_filenames]
    mfccs_stacked = torch.cat(mfccs, dim=2)
    mean = mfccs_stacked.mean(dim=2, keepdim=True)
    std = mfccs_stacked.std(dim=2, keepdim=True)
    return (mean, std)

mean, std = calculate_average_mfccs(AUDIO_DIR, mfcc_spectrogram)
# print(f"Mean: {mean}\n std: {std}")


In [23]:
def normalize_mfcc(mfcc, mean, std, epsilon=1e-8):
    return (mfcc - mean) / (std + epsilon)

def train_audio_transforms(waveform, mfcc_params):
    mean = mfcc_params["mean"]
    std = mfcc_params["std"]
    spectrogram = mfcc_params["transform"](waveform)
    return normalize_mfcc(spectrogram, mean, std)

def data_processing(data, text_transform, mfcc_params, data_type="train"):
    spectrograms = []
    labels = []
    input_lengths = []
    label_lengths = []
    
    for waveform, _, utterance, _ in data:
        if data_type == "train":
            spec = train_audio_transforms(waveform, mfcc_params).squeeze(0).transpose(0, 1)
        elif data_type == "valid":
            spec = train_audio_transforms(waveform, mfcc_params).squeeze(0).transpose(0, 1)
        else:
            raise Exception("data_type should be train or valid")
        
        spectrograms.append(spec)
        label = torch.Tensor(text_transform.text_to_int(utterance.lower()))
        labels.append(label)
        
        input_lengths.append(spec.shape[0] // 2)
        label_lengths.append(len(label))
    
    spectrograms = (
        nn.utils.rnn.pad_sequence(spectrograms, batch_first=True)
        .unsqueeze(1)
        .transpose(2, 3)
    ).squeeze(1)
    labels = nn.utils.rnn.pad_sequence(labels, batch_first=True)

    return spectrograms, labels, input_lengths, label_lengths

In [24]:
class IterMeter(object):
    """keeps track of total iterations"""

    def __init__(self):
        self.val = 0

    def step(self):
        self.val += 1

    def get(self):
        return self.val

def GreedyDecoder(
    output, labels, label_lengths, text_transform, blank_label=77, collapse_repeated=True,
):
    arg_maxes = torch.argmax(output, dim=2)
    decodes = []
    targets = []
    for i, args in enumerate(arg_maxes):
        decode = []
        targets.append(
            text_transform.int_to_text(labels[i][: label_lengths[i]].tolist())
        )
        for j, index in enumerate(args):
            if index != blank_label:
                if collapse_repeated and j != 0 and index == args[j - 1]:
                    continue
                decode.append(index.item())
        decodes.append(text_transform.int_to_text(decode))
    return decodes, targets


def train(
    model,
    device,
    train_loader,
    criterion,
    optimizer,
    scheduler,
    epoch,
    iter_meter,
):
    model.train()
    data_len = len(train_loader.dataset)
    
    for batch_idx, _data in enumerate(train_loader):
        spectrograms, labels, input_lengths, label_lengths = _data
        spectrograms, labels = spectrograms.to(device), labels.to(device)
        optimizer.zero_grad()
        
        output = model(spectrograms)  # (batch, time, n_class)
        output = F.log_softmax(output, dim=2)
        output = output.transpose(0, 1)  # (time, batch, n_class)

        loss = criterion(output, labels, input_lengths, label_lengths)
        loss.backward()

        optimizer.step()
        scheduler.step()
        iter_meter.step()
        if batch_idx % 10 == 0 or batch_idx == data_len:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(spectrograms),
                    data_len,
                    100.0 * batch_idx / len(train_loader),
                    loss.item(),
                )
            )

def test(model, device, test_loader, criterion, epoch, iter_meter, text_transform):
    model.eval()
    test_loss = 0
    test_cer, test_wer = [], []
    
    with torch.no_grad():
        for i, _data in enumerate(test_loader):
            spectrograms, labels, input_lengths, label_lengths = _data
            spectrograms, labels = spectrograms.to(device), labels.to(device)

            output = model(spectrograms)  # (batch, time, n_class)
            output = F.log_softmax(output, dim=2)
            output = output.transpose(0, 1)  # (time, batch, n_class)

            loss = criterion(output, labels, input_lengths, label_lengths)
            test_loss += loss.item() / len(test_loader)

            decoded_preds, decoded_targets = GreedyDecoder(
                output.transpose(0, 1), labels, label_lengths, text_transform
            )

            print(f"Decoded targets: {decoded_targets}")
            print(f"Decoded predicts: {decoded_preds}\n")

            for j in range(len(decoded_preds)):
                test_cer.append(cer(decoded_targets[j], decoded_preds[j]))
                test_wer.append(wer(decoded_targets[j], decoded_preds[j]))

    avg_cer = sum(test_cer) / len(test_cer)
    avg_wer = sum(test_wer) / len(test_wer)

    print(
        "Test set: Average loss: {:.4f}, Average CER: {:4f} Average WER: {:.4f}\n".format(
            test_loss, avg_cer, avg_wer
        )
    )

In [25]:
def main(dataset, text_transform, device, mfcc_params):
    learning_rate = 0.015
    batch_size = 50
    epochs = 10
    input_dim = 13  # Number of MFCC features
    num_classes = len(text_transform.char_map)+1
    
    # Total dataset length
    dataset_length = len(dataset)
    
    # Define the split lengths
    train_length = int(0.8 * dataset_length)
    test_length = dataset_length - train_length

    # Split the dataset
    train_dataset, test_dataset = random_split(dataset, [train_length, test_length])

    print(f"Training dataset length: {len(train_dataset)}")
    print(f"Testing dataset length: {len(test_dataset)}")
    
    kwargs = {"num_workers": 1, "pin_memory": True} if device=="cuda" else {}
    
    train_loader = data.DataLoader(
        dataset=train_dataset,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=lambda x: data_processing(x, text_transform, mfcc_params, "train"),
        **kwargs,
    )
    print(f"Train Loader: {train_loader}")

    test_loader = data.DataLoader(
        dataset=test_dataset,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=lambda x: data_processing(x, text_transform, mfcc_params, "valid"),
        **kwargs,
    )
    print(f"Test Loader {test_loader}")
    
    # Model, Optimizer, Loss
    model = SpeechRecognitionModel(input_dim=input_dim, num_classes=num_classes).to(device)
    print(f"Model: {model}")
    print("Num Model Parameters", sum([param.nelement() for param in model.parameters()]))
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-6, momentum=0.9)
    ctc_loss = nn.CTCLoss(blank=77)
    
    scheduler = optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=learning_rate,
        steps_per_epoch=int(len(train_loader)),
        epochs=epochs,
        anneal_strategy="linear",
    )

    iter_meter = IterMeter()
    
    for epoch in range(1, epochs+1):
        train(model,
            device,
            train_loader,
            ctc_loss,
            optimizer,
            scheduler,
            epoch,
            iter_meter)
        test(model, device, test_loader, ctc_loss, epoch, iter_meter, text_transform)
        

In [26]:
dataset = NepaliSoundDataset(
    ANNOTATIONS_FILE_PATH, AUDIO_DIR
)

mfcc_params = {
    "transform": mfcc_spectrogram.to(device),
    "sample_rate": SAMPLE_RATE,
    "mean": mean,
    "std": std,
}

# print(f"There are {len(dataset)} items")
# instance_row = dataset[890]
# print(f"Instance : {instance_row}")
# print(f"Shape: {instance_row[0].shape}")

text_transform = TextTransform()
main(dataset, text_transform, device, mfcc_params)

Training dataset length: 1650
Testing dataset length: 413
Train Loader: <torch.utils.data.dataloader.DataLoader object at 0x149c32510>
Test Loader <torch.utils.data.dataloader.DataLoader object at 0x149c9b2f0>
Model: SpeechRecognitionModel(
  (conv1): Conv1d(13, 200, kernel_size=(11,), stride=(1,), padding=(5,))
  (bn1): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (gru): GRU(200, 256, batch_first=True, bidirectional=True)
  (bn2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=512, out_features=78, bias=True)
)
Num Model Parameters 773726
Train Epoch: 1 [0/1650 (0%)]	Loss: 40.265060


In [354]:
audio_file = "/Users/santoshpandey/Desktop/ASR/code/data/OpenSLR2/wavs/nep_0258_0119737288.wav"

waveform, sample_rate = torchaudio.load(audio_file, normalize=True)
transform = torchaudio.transforms.MFCC(sample_rate=sample_rate,
                                       n_mfcc=13,
                                       melkwargs={"n_fft": 400, "hop_length": 160, "n_mels": 23, "center": False})
mfcc = transform(waveform)
print(f"MFCC shape: {mfcc.shape}")
print(f"Mfcc: {mfcc}")

MFCC shape: torch.Size([1, 13, 1204])
Mfcc: tensor([[[-2.1362e+02, -2.1397e+02, -2.1397e+02,  ..., -2.0130e+02,
          -2.0128e+02, -2.0116e+02],
         [ 4.9686e-01, -4.9400e-07, -4.9400e-07,  ...,  1.6210e+01,
           1.7660e+01,  1.6479e+01],
         [ 4.9337e-01, -5.8453e-06, -5.8453e-06,  ...,  1.6603e+01,
           1.6818e+01,  1.7065e+01],
         ...,
         [ 3.8633e-01,  1.0954e-05,  1.0954e-05,  ...,  2.0957e+00,
           2.1011e+00,  2.6708e+00],
         [ 3.6396e-01, -1.1933e-05, -1.1933e-05,  ...,  2.2543e-01,
           9.4376e-01,  5.8319e-01],
         [ 3.3991e-01, -1.1608e-05, -1.1608e-05,  ...,  6.5242e-01,
           1.3968e-01,  3.8388e-01]]])


In [355]:
audio_file = "/Users/santoshpandey/Desktop/ASR/code/data/OpenSLR2/wavs/nep_0258_0119737288.wav"
waveform, sample_rate = torchaudio.load(audio_file, normalize=True)
transform = torchaudio.transforms.MelSpectrogram(sample_rate=SAMPLE_RATE, n_fft=1024, hop_length=512, n_mels=64)
spectrogram = transform(waveform)
print(f"spectrogram shape: {spectrogram.shape}")
print(f"spectrogram: {spectrogram}")

spectrogram shape: torch.Size([1, 64, 377])
spectrogram: tensor([[[2.7597e-04, 2.6752e-05, 7.5394e-06,  ..., 5.8833e-02,
          1.9726e-01, 3.5975e-01],
         [1.6730e-04, 1.9157e-05, 9.5878e-06,  ..., 3.0364e-03,
          1.4986e-02, 3.3401e-02],
         [1.1796e-05, 6.1857e-06, 7.6325e-06,  ..., 4.9778e-04,
          4.8290e-04, 4.0218e-04],
         ...,
         [2.2947e-07, 1.8024e-07, 1.7291e-07,  ..., 1.0671e-04,
          8.7168e-05, 1.0945e-04],
         [2.4681e-07, 1.9094e-07, 1.3441e-07,  ..., 8.3326e-05,
          5.5303e-05, 7.1069e-05],
         [2.1231e-07, 1.2688e-07, 7.5778e-08,  ..., 8.9468e-05,
          4.9146e-05, 4.6855e-05]]])
